# JPA Turn Off Script

This notebook contains the script to set all JPA ports in presto to 0.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math
import presto
from presto import lockin, utils, hardware
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode

In [3]:
# Network settings for Presto Hardware
ADDRESS = '130.237.35.90'   # IP Address
PORT    = 42873             # TCP Port
Box     = 'Presto DELTA'    # Model Identifier

# Port Assignments
input_port   = 1 # Output from JPA
output_port  = 1 # Signal to JPA (vacuum for correlation experiments)
flux_port    = 5 # Pump frequency comb 
bias_port    = 4 # DC Bias for optimal operating point of JPA (based on McD curve)

In [ ]:
# Instantiate lockin device
with lockin.Lockin(address=ADDRESS,
                   port=PORT,
                   adc_mode=AdcMode.Mixed,
                   adc_fsample=AdcFSample.G2,
                   dac_mode=[DacMode.Mixed02, DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02],
                   dac_fsample=[DacFSample.G6, DacFSample.G10, DacFSample.G6, DacFSample.G6],
                   ) as lck:

    # DAC current
    lck.hardware.set_dac_current(output_port, 0)

    # Set DC bias value
    lck.hardware.set_dc_bias(0, bias_port)
    lck.hardware.sleep(1.0, False) # Let the system reach steady state

    # Configure mixer for the JPA pump
    lck.hardware.configure_mixer(freq=fNCO_pump,
                                 out_ports=flux_port,
                                 sync=False,
                                 )

    # Configure mixer just to be able to create output and input groups
    lck.hardware.configure_mixer(freq=fNCO,
                                 in_ports=input_port,
                                 out_ports=output_port,
                                 sync=True,
                                 )
    
    # Create output group for the pump
    og_pump = lck.add_output_group(ports=flux_port, nr_freq=1)
    og_pump.set_amplitudes(0)    # Set amplitude of pump = 0.04 FS
    og_pump.set_frequencies(0)
    og_pump.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)
    
    # Create output group for the signal
    og = lck.add_output_group(ports=output_port, nr_freq=1)
    og.set_amplitudes(0)
    og.set_frequencies(0)
    og.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)
    
    # Create input group for measurement
    ig = lck.add_input_group(port=input_port, nr_freq=1)
    ig.set_frequencies(0)
    og.set_amplitudes(0)